In [1]:
import h5py
import torch
import torch.nn as nn
import random


import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import datetime
from tempfile import TemporaryFile
from scipy.io import loadmat
from scipy.io import savemat
from torch.utils.data import DataLoader, TensorDataset

sys.path.append('../helper')
import loader
import utils 

sys.path.append('../train/Pix2Pix/helperPix2Pix')
from generator_model import Generator
from discriminator_model import Discriminator

In [2]:
rowss = '3500_3516'
gan_model_path = '../model/static/GAN/BS16/'+ rowss + '/ver11_/'
cnn_model_path = '../model/static/CNN/BS16/'+ rowss + '/ver27_/'
CDL_path = '../../CDL Customization/Data/ver39_/'
snr = 5
device = 'cpu'

# Min-max Scaler to [-1 1] range
norm_approach = 'minmax'
lower_range = -1
CNN_DropOut = 0.2
CNN_activation = 'Tanh'

gan_LS_G = Generator()
checkpoint_LS_G = torch.load(gan_model_path + str(snr) + 'dB/G_1_LS_GAN_model.pth')
gan_LS_G.load_state_dict(checkpoint_LS_G['model_state_dict'])
gan_LS_G.eval()

gan_LI_G = Generator()
checkpoint_LI_G = torch.load(gan_model_path + str(snr) + 'dB/G_1_LS_LI_GAN_model.pth')
gan_LI_G.load_state_dict(checkpoint_LI_G['model_state_dict'])
gan_LI_G.eval()


model_LS_CNN = utils.CNN_Est(dropOut=0, act =CNN_activation)
checkpoint_LS = torch.load(cnn_model_path + str(snr) + 'dB/CNN_1_LS_CNN_model.pth')
model_LS_CNN.load_state_dict(checkpoint_LS['model_state_dict'])
model_LS_CNN.eval()

model_LS_LI_CNN = utils.CNN_Est(dropOut=CNN_DropOut, act =CNN_activation)
checkpoint_LI = torch.load(cnn_model_path + str(snr) + 'dB/CNN_1_LS_LI_CNN_model.pth')
model_LS_LI_CNN.load_state_dict(checkpoint_LI['model_state_dict'])
model_LS_LI_CNN.eval()


/home/thien/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


FileNotFoundError: [Errno 2] No such file or directory: '../model/static/CNN/BS16/3500_3516/ver27_/5dB/CNN_1_LS_CNN_model.pth'

In [ ]:

mapBaseData_path = CDL_path + str(snr) + 'dB/1_mapBaseData.mat'
file = h5py.File(mapBaseData_path, 'r')
print('Size of data:')
print(file['H_data'].shape)

H_true = np.empty((0, 2, 612, 14)) # true channel
H_equal = np.empty((0, 2, 612, 14)) # noisy channel # LS channel
H_linear = np.empty((0, 2, 612, 14)) # noisy channel # LS+Linear Interpolated channel
        
H_true = np.concatenate((H_true, (np.array(file['H_data']))) , axis = 0) # N_samples x channel(2) x height(614) x width(14)
H_equal = np.concatenate((H_equal, (np.array(file['H_equalized_data']))), axis = 0)
H_linear = np.concatenate((H_linear, (np.array(file['H_linear_data']))), axis=0)

H_true = torch.from_numpy(H_true)
H_equal = torch.from_numpy(H_equal)
H_linear = torch.from_numpy(H_linear)

testData_LS_normd, test_LS_min, test_LS_max = utils.minmaxScaler(H_equal)
testData_LI_normd, test_LI_min, test_LI_max = utils.minmaxScaler(H_linear)

min_maxLS = torch.stack((test_LS_min, test_LS_max), dim=1)
min_maxLI = torch.stack((test_LI_min, test_LI_max), dim=1)

testData_LS_normd = testData_LS_normd.to(device, dtype=torch.float)
testData_LI_normd = testData_LI_normd.to(device, dtype=torch.float)
min_maxLS         = min_maxLS.to(device, dtype=torch.float)
min_maxLI         = min_maxLI.to(device, dtype=torch.float)
                                    

H_true = H_true.to(device, dtype=torch.float)

In [ ]:
idx = random.randint(0, file['H_data'].shape[0]-1)